In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [2]:
import os
import json
import shutil

## Single task IVT from Rendezvous

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import os
import json
import shutil

from utils.general.dataset_variables import TripletSegmentationVariables

triplet_dict = TripletSegmentationVariables.categories['triplet']

In [2]:
# latest 
# experiment = 'rendezvous_cholectriplet_seg'
# first_stage_results_source_folder = "../../datasets/my_triplet_seg_datasets/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/ann_dir"
# first_stage_results_destination_folder = f"../resnet_model/work_dirs/{experiment}/combine_first_and_second_stage_results"
# second_stage_rendezvous_results_file = f"../resnet_model/work_dirs/{experiment}/results_rendezvous_l8_cholectcholect50-tripletseg_k0_batchnorm_lowres.json"
# updated_second_stage_rendezvous_results_file = f"../resnet_model/work_dirs/{experiment}/updated_rendezvous_results.json"

experiment = 'rendezvous_cholectriplet_seg'
first_stage_results_source_folder = "../../datasets/my_triplet_seg_datasets/triplet_segmentation_dataset_v3/mask2former_instrument_prediction/ann_dir"
first_stage_results_destination_folder = f"../resnet_model/results_from_dgx/v3/{experiment}/combine_first_and_second_stage_results"

second_stage_rendezvous_results_file = f"../resnet_model/results_from_dgx/v3/{experiment}/results_rendezvous_l8_cholectcholect50-tripletseg_k0_batchnorm_lowres.json"
updated_second_stage_rendezvous_results_file =f"../resnet_model/results_from_dgx/v3/{experiment}/updated_rendezvous_results.json"

In [5]:
shutil.copytree(first_stage_results_source_folder, first_stage_results_destination_folder)

'../resnet_model/results_from_dgx/v3/rendezvous_cholectriplet_seg/combine_first_and_second_stage_results'

In [3]:
triplet_dict

OrderedDict([('1', 'grasper,dissect,cystic_plate'),
             ('2', 'grasper,dissect,gallbladder'),
             ('3', 'grasper,dissect,omentum'),
             ('4', 'grasper,grasp,cystic_artery'),
             ('5', 'grasper,grasp,cystic_duct'),
             ('6', 'grasper,grasp,cystic_pedicle'),
             ('7', 'grasper,grasp,cystic_plate'),
             ('8', 'grasper,grasp,gallbladder'),
             ('9', 'grasper,grasp,gut'),
             ('10', 'grasper,grasp,liver'),
             ('11', 'grasper,grasp,omentum'),
             ('12', 'grasper,grasp,peritoneum'),
             ('13', 'grasper,grasp,specimen_bag'),
             ('14', 'grasper,pack,gallbladder'),
             ('15', 'grasper,retract,cystic_duct'),
             ('16', 'grasper,retract,cystic_pedicle'),
             ('17', 'grasper,retract,cystic_plate'),
             ('18', 'grasper,retract,gallbladder'),
             ('19', 'grasper,retract,gut'),
             ('20', 'grasper,retract,liver'),
             ('21

In [22]:
import numpy as np
import os
import json
def combine_rendezvous_with_segmentation_labels(first_stage_results_folder, rendezvous_results_file):
    """
    Combines instrument segmentation labels (first stage) with verb/target predictions (second stage).
    - Matches instruments by their class names.
    - Prints successful and failed matches.
    - Counts images where ALL annotations were successfully matched.
    Args:
        first_stage_results_folder (str): Path to folder containing first-stage segmentation JSON files.
        rendezvous_results_file (str): Path to the JSON file containing updated Rendezvous results.
    """
    # Load the second-stage Rendezvous predictions
    with open(rendezvous_results_file, 'r') as f:
        rendezvous_results = json.load(f)
    total_images = 0  # Total images processed
    fully_matched_images = 0  # Count of images where all annotations were successfully matched
    # Iterate through all first-stage JSON files
    for file_name in os.listdir(first_stage_results_folder):
        file_name_no_ext = file_name.split('.')[0]
        if file_name.endswith(".json"):
            first_stage_path = os.path.join(first_stage_results_folder, file_name)
            # Load the first-stage JSON (segmentation labels)
            with open(first_stage_path, 'r') as f:
                first_stage_data = json.load(f)
            total_images += 1  # Increment processed images count
            all_annotations_matched = True  # Assume all will be matched
            
            file_name_in_result_dict = file_name_no_ext.replace('t50_', '') 
            # Get corresponding triplet predictions from Rendezvous
            if file_name_in_result_dict in rendezvous_results:
                print(f'found {file_name_no_ext}')
                triplet_probs = np.array(rendezvous_results[file_name_in_result_dict]["triplet_prediction"])
                valid_triplets =  np.where(triplet_probs>0.5)[0]
                # print(triplet_probs)
                # print(valid_triplets)
                triplet_predictions = [ triplet_dict[str(valid_triplet+1)] for valid_triplet in valid_triplets]  # get triplet names 
                # print(triplet_predictions) 
                # raise ValueError('wait')
            else:
                raise ValueError('{file_name_in_result_dict} not in result dict')
            # Extract predicted instrument-verb-target triplets
            predicted_triplets = {}
            for triplet in triplet_predictions:
                instrument, verb, target = triplet.split(',')
                if instrument not in predicted_triplets:
                    predicted_triplets[instrument] = []
                predicted_triplets[instrument].append((verb, target))
            # Iterate through each shape (segmented instrument) in the first-stage JSON
            # Filter and update shapes
            matched_shapes = []
            for shape in first_stage_data['shapes']:
                instrument_label = shape['label']  # Instrument name from segmentation
                if instrument_label in predicted_triplets:
                    # Assign the first detected verb-target pair
                    verb_name, target_name = predicted_triplets[instrument_label][0]
                    shape['verb'] = verb_name
                    shape['target'] = target_name
                    print(f"[SUCCESS] Matched instrument '{instrument_label}' in {file_name_no_ext} → ({verb_name}, {target_name})")
                    matched_shapes.append(shape)
                else:
                    # matched_shapes.append(shape) # remove this to not have any shape not matched
                    print(f"[FAILURE] No match for instrument '{instrument_label}' in {file_name_no_ext}")
                    all_annotations_matched = False  # At least one failed match, so the image is not fully matched
            # Update shapes with only matched ones
            first_stage_data['shapes'] = matched_shapes
            # Increment fully matched image count if all annotations were matched
            if all_annotations_matched:
                fully_matched_images += 1
            # Save the updated JSON back to the folder
            with open(first_stage_path, 'w') as f:
                json.dump(first_stage_data, f, indent=4)
    print("\n==== Summary ====")
    print(f"Total Images Processed: {total_images}")
    print(f"Images where ALL annotations were successfully matched: {fully_matched_images}")
    print(f"Images with at least one failed match: {total_images - fully_matched_images}")
    print("\nProcessing complete. Check the output for matching results.")


In [23]:
combine_rendezvous_with_segmentation_labels(first_stage_results_destination_folder, updated_second_stage_rendezvous_results_file)

found t50_VID14_000000
found t50_VID14_000001
[FAILURE] No match for instrument 'grasper' in t50_VID14_000001
found t50_VID14_000002
found t50_VID14_000003
found t50_VID14_000004
found t50_VID14_000005
found t50_VID14_000006
found t50_VID14_000007
found t50_VID14_000008
found t50_VID14_000009
found t50_VID14_000010
found t50_VID14_000011
[SUCCESS] Matched instrument 'grasper' in t50_VID14_000011 → (retract, gallbladder)
found t50_VID14_000012
[SUCCESS] Matched instrument 'grasper' in t50_VID14_000012 → (grasp, gallbladder)
found t50_VID14_000013
[SUCCESS] Matched instrument 'grasper' in t50_VID14_000013 → (retract, gallbladder)
found t50_VID14_000014
[SUCCESS] Matched instrument 'grasper' in t50_VID14_000014 → (retract, gallbladder)
found t50_VID14_000015
[FAILURE] No match for instrument 'bipolar' in t50_VID14_000015
[FAILURE] No match for instrument 'grasper' in t50_VID14_000015
found t50_VID14_000016
[FAILURE] No match for instrument 'hook' in t50_VID14_000016
found t50_VID14_000017